In [1]:
import pandas as pd
import numpy as np 

train = pd.read_csv("../1. Original Data/train.csv")
test = pd.read_csv("../1. Original Data/test.csv")
train.info()
#print (test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [2]:
def cleaning (data):
    
    # Fist of all dropping null values in Embarked only 2 records
    data.dropna(subset=['Embarked'],inplace = True)
    # computing null in Age * Fare columns (continuous data)
    data['Fare'] = data['Fare'].fillna(data['Fare'].dropna().median())
    data['Age'] = data['Age'].fillna(data['Age'].dropna().median())
    
    #-----------------------Feature Engineering------------------- 
    #Creating Famaly Size column
    data['FamilySize']=data['SibSp']+data['Parch']+1
    #Creating cabin letter calumn
    data['Cabin_letter'] = data['Cabin'].apply(lambda x: str(x)[0])
    #Creating Name title 
    data['Name_title'] = data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
    #Create column is ticket numeric or not
    data['Numeric_ticket'] = data['Ticket'].apply(lambda x: 1 if x.isnumeric() else 0)
    #-----------------------Encoding Feature data-------------------
    # By gender 
    from sklearn.preprocessing import LabelEncoder 
    labelencoder = LabelEncoder()
    data["Sex"]= labelencoder.fit_transform(data["Sex"])
    # By Embarked 
    data["Embarked"]= labelencoder.fit_transform(data["Embarked"])
    #By Name_title
    data["Name_title"]= labelencoder.fit_transform(data["Name_title"])
    #By Cabin_letter
    data["Cabin_letter"]= labelencoder.fit_transform(data["Cabin_letter"])
    #--------------------Creating Age Bins------------------------
    bins = [0, 5, 10, 15, 20, 25, 30,35,40,45,50,55,60,65,70,75,80,85]
    labels = [0, 5, 10, 15, 20, 25, 30,35,40,45,50,55,60,65,70,75,80]
    data['Age_bins'] = pd.cut(data['Age'], bins=bins, labels=labels)
    #--------------------------Droping values-----------------------
    feature_drop = ["Name","Cabin","Ticket","Age"]
    data.drop(feature_drop, axis = 1, inplace= True)
    #-----------------------Some scaling proceses------------------
    data['Fare'] = np.log(data['Fare']+1)
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    data[['Age_bins','SibSp','Parch','Fare','FamilySize',"Name_title","Cabin_letter"]] = scaler.fit_transform(data[[
        'Age_bins','SibSp','Parch','Fare','FamilySize',"Name_title","Cabin_letter"]])

In [3]:
cleaning(train)
cleaning(test)
train.to_csv('../2. Prepared Data/train_cleaned.csv',index = False)
test.to_csv('../2. Prepared Data/test_cleaned.csv',index = False)